In [1]:
import os
cwd = os.getcwd()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from lib.eeg_functions import *
from lib.biggan_functions import *
from scipy.fftpack import fft, rfft, fftfreq, irfft, ifft, rfftfreq
from scipy import signal

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Reading Data

### Options for EEG

In [3]:
# Define options
opt = {}

# Filtering options
opt['detrend'] = True
opt['ft-low'] = 0 #15
opt['ft-high'] = 257 #70

### Reading EEG

In [4]:
# Defining modifiable parameters and options
OPTION = 5
NUM_INTERVALS = 40
NUM_SAMPLES_PER_INTERVAL = 20
CHUNK_SIZE_CHOSEN = 257

Z_LENGTH = 120
Y_LENGTH = 40 # 1000


# EEG chunks for each image
EEG_FILENAME = 'data/averaged_eeg_chunks.csv'

NUM_IMAGES = NUM_INTERVALS * NUM_SAMPLES_PER_INTERVAL
separated_chunks_array = read_eeg_from_file(EEG_FILENAME, NUM_IMAGES, CHUNK_SIZE_CHOSEN)
print('Separated chunks:', separated_chunks_array.shape)


# Latent representation of each image
FILENAME = 'data/df_latent_values.csv'
DF_LATENT_VALUES_READ = pd.read_csv(FILENAME, index_col = 0)
DF_LATENT_VALUES_READ = DF_LATENT_VALUES_READ[:NUM_IMAGES] # selecting subset if not using all
print('Read latent values df:', DF_LATENT_VALUES_READ.shape)


bgf = biggan_functions()
# Convert latent representation to array for NN
latent_z, latent_y = bgf.convert_latent_to_array(Z_LENGTH, Y_LENGTH, DF_LATENT_VALUES_READ)
print('latent z:', latent_z.shape, 'latent y:', latent_y.shape)

# We are folding each image data into layer of frames. Thus, one image is present only in one 3D convolution, 
# does not span across multiple structures
NUM_FRAMES = 6
FRAME_SIZE = int(CHUNK_SIZE_CHOSEN/NUM_FRAMES)

Separated chunks: (800, 257, 35)
Read latent values df: (800, 1123)
Set success
latent z: (800, 120) latent y: (800, 40)


In [5]:
eeg = separated_chunks_array
a,b,c = eeg.shape

# Standardise
mean = np.mean(eeg, 0)
stddev = np.std(eeg, 0)
eeg = (eeg - mean)/stddev
print(eeg.shape, latent_y.shape, latent_z.shape)

(800, 257, 35) (800, 40) (800, 120)


In [6]:
# convert 35 channels to 32 channels
# another option is to use linear embedding for dimension consistency
_1_eeg = np.array([np.vstack((np.ones((1,35)), eeg[i])) for i in range(800)])
avg_first_3 = np.mean(_1_eeg[:,:,0:3], 2, keepdims=True)
avg_last_2 = np.mean(_1_eeg[:,:,-2:], 2, keepdims=True)
avg_eeg = np.array([np.hstack((avg_first_3[i], _1_eeg[i,:,3:-2])) for i in range(800)])
avg_eeg = np.array([np.hstack((avg_eeg[i], avg_last_2[i])) for i in range(800)])
print(avg_eeg.shape)

(800, 258, 32)


In [7]:
# Time axis
a,b,c = avg_eeg.shape
N = b
T = 1.0/1000.0
time = np.linspace(0.0, N*T, N)

# Frequency axis
w = rfftfreq(N, T)
# FFT
avg_eeg_fft = rfft(avg_eeg)

# Notch Filter: 50 Hz
avg_eeg_fft[:,np.bitwise_and(w > 49, w < 51)] = 0

if opt['detrend']:
    avg_eeg = irfft(avg_eeg_fft)
    avg_eeg = signal.detrend(avg_eeg,axis=1)
# Passband filter
if opt['ft-high']:
    avg_eeg_fft[:,w > opt['ft-high']] = 0
if opt['ft-low']:
    avg_eeg_fft[:,w < opt['ft-low']] = 0
    avg_eeg = avg_irfft(eeg_fft)

avg_eeg = torch.tensor(avg_eeg) #.t()
print(avg_eeg.shape)

torch.Size([800, 258, 32])


### Splitting Training and Testing

In [8]:
ids = np.random.permutation(a)
idx = ids[:int(a/5)]

train_eeg = avg_eeg[np.setdiff1d(ids, idx), :, :]
train_latent_y = latent_y[np.setdiff1d(ids, idx), :]
train_latent_z = latent_z[np.setdiff1d(ids, idx), :]

test_eeg = avg_eeg[idx, :, :]
test_latent_y = latent_y[idx, :]
test_latent_z = latent_z[idx, :]

print(train_eeg.shape,train_latent_y.shape,train_latent_z.shape)
print(test_eeg.shape, test_latent_y.shape, test_latent_z.shape)

torch.Size([640, 258, 32]) (640, 40) (640, 120)
torch.Size([160, 258, 32]) (160, 40) (160, 120)


### Window shifting

In [9]:
shift = 20
# larger window_size, less intersection
train_src = train_eeg[:50,:-shift,:]
train_tgt = train_eeg[:50,shift:,:]
print(train_src.shape, train_tgt.shape)

test_src = test_eeg[:10,:-shift,:]
test_tgt = test_eeg[:10,shift:,:]
print(test_src.shape, test_tgt.shape)

torch.Size([50, 238, 32]) torch.Size([50, 238, 32])
torch.Size([10, 238, 32]) torch.Size([10, 238, 32])


## Model Architecture

### Options for Model

In [10]:
# # Model options
# opt['lstm-layers'] = 1
# opt['lstm-size'] = 35 # 128
# opt['embedding-size'] = 35 # 128
# opt['num-classes'] = 40

In [11]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)


In [12]:
class Generator(nn.Module):
    "Define standard linear"
    def __init__(self, d_model):
        super(Generator, self).__init__()
        self.proj1 = nn.Linear(d_model, 128)
        self.proj2 = nn.Linear(128, d_model)

    def forward(self, x):
        return self.proj2(self.proj1(x))

## Encoder and Decoder Stacks   

### Encoder

In [13]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [14]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [15]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [16]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder

In [17]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [18]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [19]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

### Attention      
$$                                                                         
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V               
$$   

In [20]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

$$    
\mathrm{MultiHead}(Q, K, V) = \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O    \\                                           
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)                                
$$                                                                                                                 

Where the projections are parameter matrices $W^Q_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in \mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$.   

In [21]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

### Positionwise Forward
$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$  

In [22]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Optional

In [23]:
# Later
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        hh =self.lut(x) * math.sqrt(self.d_model)
        return self.lut(x) * math.sqrt(self.d_model)

## Positional Encoding           
In this work, we use sine and cosine functions of different frequencies:       

$$PE_{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}})$$   

In [24]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0.0, max_len).unsqueeze(1) # [max_len,1] 
        div_term = torch.exp(torch.arange(0.0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

# Full Model

In [36]:
# Model options
opt['Transformer-layers'] = 3
opt['Model-dimensions'] = 32
opt['feedford-size'] = 2048 
opt['headers'] = 8
opt['dropout'] = 0.1

In [25]:
def make_model(N=6, d_model=32, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(c(position)),
        nn.Sequential(c(position)),
        Generator(d_model))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

#  Optimizor
$$
lrate = d_{\text{model}}^{-0.5} \cdot                                                                                                                                                                                                                                                                                                
  \min({step\_num}^{-0.5},                                                                                                                                                                                                                                                                                                  
    {step\_num} \cdot {warmup\_steps}^{-1.5})                                                                                                                                                                                                                                                                               
$$                     

In [26]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model_size=32, factor=2, warmup=4000,
        optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

## Loss Computation

In [27]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt

    def __call__(self, x, y, norm): 
        x = self.generator(x)
        loss = self.criterion(x.contiguous(), y.contiguous()) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm

## Training

In [28]:
# Padding Mask for src, Sequence Mask for tgt
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask =  Variable(torch.ones(src.size(0), 1, src.size(1)) )
        if trg is not None:
            self.trg = trg[:, :-1,:]
            self.trg_y = trg[:, 1:,:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = self.trg_y.size(1)
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
 
        tgt_mask = Variable(torch.ones(tgt.size(0), 1, tgt.size(1),dtype = torch.long))
        tgt_mask = tgt_mask.type_as(tgt_mask.data) & Variable(subsequent_mask(tgt.size(1)).type_as(tgt_mask.data))
        return tgt_mask

In [29]:
def data_gen(nbatches,src,tgt):
    for i in range(nbatches):
        src = Variable(src.float(), requires_grad=False)
        tgt = Variable(tgt.float(), requires_grad=False)
        yield Batch(src, tgt, 0)

In [35]:
def run_epoch(data_iter, model, loss_compute):
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
    return total_loss / total_tokens

In [ ]:
criterion = nn.MSELoss()
model = make_model(opt['Transformer-layers'],opt['Model-dimensions'],opt['feedford-size'],opt['headers'],opt['dropout'])
model_opt = NoamOpt(model_size=opt['Model-dimensions'], factor=1, warmup=400,
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9))

total_epoch = 20
for epoch in range(total_epoch):
    model.train()
    loss1 = run_epoch(data_gen(1,train_src,train_tgt), model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
    
    model.eval()
    loss2 = run_epoch(data_gen(1,test_src,test_tgt), model, 
            SimpleLossCompute(model.generator, criterion, None))
    print('Epoch[{}/{}], train_loss: {:.6f}, test_loss: {:.6f}'
              .format(epoch+1, total_epoch, loss1,loss2))

Epoch[1/20], train_loss: 0.002964, test_loss: 0.002772
Epoch[2/20], train_loss: 0.002838, test_loss: 0.002543
Epoch[3/20], train_loss: 0.002614, test_loss: 0.002245
Epoch[4/20], train_loss: 0.002336, test_loss: 0.001902
Epoch[5/20], train_loss: 0.002019, test_loss: 0.001529
Epoch[6/20], train_loss: 0.001693, test_loss: 0.001164
Epoch[7/20], train_loss: 0.001369, test_loss: 0.000865
Epoch[8/20], train_loss: 0.001094, test_loss: 0.000668
Epoch[9/20], train_loss: 0.000924, test_loss: 0.000555
Epoch[10/20], train_loss: 0.000812, test_loss: 0.000494
Epoch[11/20], train_loss: 0.000752, test_loss: 0.000467
Epoch[12/20], train_loss: 0.000716, test_loss: 0.000455
Epoch[13/20], train_loss: 0.000693, test_loss: 0.000433
Epoch[14/20], train_loss: 0.000663, test_loss: 0.000393
Epoch[15/20], train_loss: 0.000618, test_loss: 0.000344
Epoch[16/20], train_loss: 0.000556, test_loss: 0.000296
Epoch[17/20], train_loss: 0.000499, test_loss: 0.000250
Epoch[18/20], train_loss: 0.000441, test_loss: 0.000207
